
*Note:* You can run this example locally (`ploomber examples -n python-api`) or in Binder (hosted JupyterLab ).

[![binder-logo](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/ploomber/binder-env/main?urlpath=git-pull%3Frepo%3Dhttps%253A%252F%252Fgithub.com%252Fploomber%252Fprojects%26urlpath%3Dlab%252Ftree%252Fprojects%252Fpython-api%252FREADME.ipynb%26branch%3Dmaster)


# Python API

Pipeline project using the Python API.

If you're new to the Python API, check out [python-api-examples/](../python-api-examples) directory, containing tutorials and more examples.

## Setup

~~~bash
# if using conda
conda env create --file environment.yml
conda activate python-api

# otherwise use pip directly
pip install -r requirements.txt
# install the pipeline as a package in editable mode
pip install --editable .
~~~

## Description

This pipeline has three tasks:

1. Load task (Python function): CSV file
2. Clean task (Python script):  Jupyter notebook and another CSV file
3. Plot task (Python scripts): Jupyter notebook

## Build

In [1]:
%%sh
ploomber build

name    Ran?      Elapsed (s)    Percentage
------  ------  -------------  ------------
load    True         0.007502       0.14093
clean   True         2.21771       41.6611
plot    True         3.09801       58.198


Building task 'plot': 100%|██████████| 3/3 [00:08<00:00,  2.73s/it]
